## response collection

The purpose of this notebook is to compare ys predicted at a site with with other sources of information about that site.


In [ ]:
import typing as t
from dataclasses import dataclass
from pathlib import Path
from datetime import datetime, timedelta

import httpx
from pymongo import MongoClient

DB_NAME = "validation_data"
COLLECTION_NAME = "api_response"

coordinates = (-30.2466, -70.7494)
lat, lon = coordinates


@dataclass
class Site:
    lat: str
    lon: str
    astro_twilight_datetime: datetime


def get_db():
    with open(Path.cwd() / ".env") as f:
        first_line, *_ = f.read().split("\n")
        idx = first_line.find("=")
        uri = first_line[idx + 1 :]

    client = MongoClient(uri)
    return client[DB_NAME]


db = get_db()
db

In [ ]:
collection = db[COLLECTION_NAME]
collection

In [93]:
async def get_prediction():
    async with httpx.AsyncClient() as client:
        url = f"http://localhost:8000/api/prediction?lat={lat}&lon={lon}"
        r = await client.get(url)
        return r.json()


async def insert_prediction():
    prediction = await get_prediction()
    collection.insert_one(prediction)
    return prediction


prediction = await insert_prediction()
prediction

{'brightness_mpsas': 19.7862,
 'astro_twilight': {'iso': '2024-01-01 01:24:21.890', 'type': 'nearest'},
 '_id': ObjectId('6591e8a0b29b8604829db6d0')}

In [ ]:
open_meteo_url = "https://archive-api.open-meteo.com/v1"


async def get_historical_meteo_data_at_site(site: Site) -> t.Any:
    def format_datetime(dt: datetime):
        return dt.strftime("%Y-%m-%d")

    astro_twilight_plus_one = site.astro_twilight_datetime + timedelta(days=0)
    start_date_str = format_datetime(site.astro_twilight_datetime)
    end_date_str = format_datetime(astro_twilight_plus_one)
    url = f"{open_meteo_url}/archive?latitude={site.lat}&longitude={site.lon}&start_date={start_date_str}&end_date={end_date_str}&hourly=temperature_2m,cloud_cover"
    print(url)
    async with httpx.AsyncClient() as client:
        r = await client.get(url)
        r.raise_for_status()
        json = r.json()
        return json["hourly"]["cloud_cover"][site.astro_twilight_datetime.hour]

In [ ]:
items_in_collection = list(collection.find())
items_in_collection

In [ ]:
for item in items_in_collection:
    astro_twilight_iso = item["astro_twilight"]["iso"]
    astro_twilight_datetime = datetime.strptime(
        astro_twilight_iso, "%Y-%m-%d %H:%M:%S.%f"
    )
    s = Site(
        lat=lat,
        lon=lon,
        astro_twilight_datetime=astro_twilight_datetime,
    )
    meteo_data_at_site = await get_historical_meteo_data_at_site(s)
    print(meteo_data_at_site)

In [ ]:
collection.delete_many({})